In [1]:
#python -m pip install opencv3

In [ ]:
import cv2
import numpy as np

from matplotlib import pyplot as plt

%matplotlib inline


import os

from IPython.display import clear_output

%matplotlib inline

### face detection

- scaleFactor – Parameter specifying how much the image size is reduced at each image scale.


- min_neighbors-Parameter specifying how many neighbors each candidate rectangle should have to retain it.


- minSize – Minimum possible object size. Objects smaller than that are ignored.


- maxSize – Maximum possible object size. Objects larger than that are ignored.

In [ ]:

faceDetect=cv2.CascadeClassifier('xml/frontal_face.xml')
cam = cv2.VideoCapture(1)

cv2.namedWindow("Face", cv2.WINDOW_AUTOSIZE)

while True:
    ret,img = cam.read()
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = faceDetect.detectMultiScale(gray,scaleFactor=1.3,minNeighbors=3)

    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)

    
    cv2.imshow("Face",img)
    if (cv2.waitKey(10)== ord('q')):
        break

cam.release()
cv2.destroyAllWindows()

#scaleFactor – Parameter specifying how much the image size is reduced at each image scale.
#min_neighbors-Parameter specifying how many neighbors each candidate rectangle should have to retain it.
#minSize – Minimum possible object size. Objects smaller than that are ignored.
#maxSize – Maximum possible object size. Objects larger than that are ignored.

<h2 align="center" style='color: #be2830'>Build Our Dataset</h2>
<h4 align="center">
Detect $\rightarrow$ Cut $\rightarrow$ Normalize $\rightarrow$ Resize $\rightarrow$ Save</h4>

### detect face


In [ ]:

def detect_face(frame):
    detector = cv2.CascadeClassifier("xml/frontal_face.xml")

    
    #faces = detector.detectMultiScale(frame,scaleFactor=1.3,minNeighbors=5)
    scale_factor = 1.2
    min_neighbors =5
    min_size = (30,30)
    flags = cv2.CASCADE_SCALE_IMAGE

    faces = detector.detectMultiScale(frame,scaleFactor=scale_factor,
                                     minNeighbors=min_neighbors,
                                     minSize=min_size,
                                     flags=flags)
    
    return faces

### cut face

In [ ]:
def cut_faces(image, faces_coord):
    faces = []
      
    for (x, y, w, h) in faces_coord:
        #w_rm = int(0.2 * w / 2)
        faces.append(image[y: y + h, x : x + w ])
         
    return faces

### normalize faces by increasing pixel intensity(brightness)

In [ ]:
def normalize_intensity(images):
    images_norm = []
    for image in images:
        is_color = len(image.shape) == 3 
        if is_color:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        images_norm.append(cv2.equalizeHist(image))
    return images_norm

### Resize face

 ### cv.INTER_AREA for shrinking & cv.INTER_CUBIC for zooming

In [ ]:
def resize(images,size=(47,62)):
    image_resize = []
    
    for image in images:
        if image.shape < size:
            img_size = cv2.resize(image,size,interpolation=cv2.INTER_CUBIC)
        else:
            img_size = cv2.resize(image,size,interpolation=cv2.INTER_AREA)
        image_resize.append(img_size)
        
    return image_resize



### make pipeline

In [ ]:
def normalize_faces(frame, faces_coord):
    faces = cut_faces(frame, faces_coord)
    faces = normalize_intensity(faces)
    
    faces = resize(faces)
    return faces

In [ ]:
def plot_show(image,title=""):
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    plt.axis("off")
    plt.title(title)
    plt.imshow(image,cmap="Greys_r")
    plt.show()

In [ ]:
def draw_rectangle(image, coords):
    for (x, y, w, h) in coords:
        #w_rm = int(0.2 * w / 2) 
        cv2.rectangle(image, (x , y), (x + w , y + h), (0,0,255),2)

In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from IPython.display import clear_output

%matplotlib inline

cam = cv2.VideoCapture(1)
#cv2.namedWindow("Face", cv2.WINDOW_AUTOSIZE)
folder = "people/"+input('Person:').lower()


if not os.path.exists(folder):
    os.mkdir(folder)
    
    flag_start_capturing = False
    sample=1
    #cam = cv2.VideoCapture(0)
    cv2.namedWindow("Face", cv2.WINDOW_AUTOSIZE)

    while True:
        ret,frame = cam.read()
        
        
        
        faces_coord = detect_face(frame)

        if len(faces_coord):
            faces = normalize_faces(frame,faces_coord)
            #faces = normalize_intensity(faces)
            cv2.imwrite(folder + '/' + str(sample)+'.jpg',faces[0])
            plot_show(faces[0],"Image saved:"+str(sample))
            clear_output(wait=True)
            
        draw_rectangle(frame,faces_coord)
        cv2.imshow('Face',frame)
        keypress=cv2.waitKey(1)
        
        if keypress == ord('c'):
            
            if flag_start_capturing == False:
                flag_start_capturing = True
            else:
                flag_start_capturing = False
                sample = 0
        
        if flag_start_capturing == True:
            sample += 1
        if sample >15:
            break

    cam.release()
    cv2.destroyAllWindows()
.else:
    print ("This name already exists.")

In [ ]:
cam.release()

In [ ]:
### datasets for other class

basepath = 'C:\\Users\\Vineet\\scikit_learn_data\\lfw_home\\lfw_funneled\\'
#C:\Users\Vineet\scikit_learn_data\lfw_home
images = os.listdir(basepath) 
print (len(images))
data = images[:210]

for i,folder in enumerate(data,start=1):
    
    files=os.listdir(basepath+'\\'+folder)
    for k,img in enumerate(files,start=1):
        if img.endswith('.jpg'):
            #print img
            frame=cv2.imread(basepath+'\\'+folder+'\\'+img,0)
        #print frame
       
            faces_coord = detect_face(frame)
            if len(faces_coord):
                faces = cut_faces(frame, faces_coord)
                #print faces
                faces = normalize_intensity(faces)
                faces = resize(faces)
                cv2.imwrite('people/unknown/' + str(i)+'.jpg',faces[0])
                
                break

### collect datasets

In [ ]:
def collect_dataset():
    images = []
    labels = []
    labels_dic = {}
    #people = [person for person in os.listdir("Male_female/")]
    people = [person for person in os.listdir("people/")]
    #people = [person for person in os.listdir("people/")]
    for i, person in enumerate(people):
        labels_dic[i] = person
        for image in os.listdir("people/" + person):
            if image.endswith('.jpg'):
                images.append(cv2.imread("people/" + person + '/' + image, 0))
                labels.append(i)
    return (images, np.array(labels), labels_dic)

In [ ]:
images, labels, labels_dic = collect_dataset()

In [ ]:
print (len(images))
print (labels_dic)

In [ ]:
X_train=np.asarray(images)

In [ ]:
X_train.shape

In [ ]:
train=X_train.reshape(len(X_train),-1)

In [ ]:
train.shape

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca1 = PCA()
pca1.fit(train)
plt.plot(np.cumsum(pca1.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')

In [ ]:
pca1 = PCA(n_components=.99)
new_train=pca1.fit_transform(train)

In [ ]:
pca1.n_components_

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV,KFold

In [ ]:
param_grid = {'C':[.0001,.001,.01,.1,1,10]}

In [ ]:
kf=KFold(n_splits=5,shuffle=True)

In [ ]:
gs_svc = GridSearchCV(SVC(kernel='linear',probability=True),param_grid=param_grid,cv=kf,scoring='accuracy')

In [ ]:
gs_svc.fit(new_train,labels)

In [ ]:
gs_svc.best_score_

In [ ]:
gs_svc.best_params_

In [ ]:
svc1 = gs_svc.best_estimator_

In [ ]:
import pickle

filename = 'svc_face.pkl'
f=open(filename, 'wb')
pickle.dump(svc1,f)
 

f.close()


In [ ]:
import pickle

filename = 'svc_face.pkl'
svc1 = pickle.load(open(filename, 'rb'))

In [ ]:
cam = cv2.VideoCapture(1)
font=cv2.FONT_HERSHEY_PLAIN
cv2.namedWindow("opencv_face", cv2.WINDOW_AUTOSIZE)



while True:
    ret,frame = cam.read()
    
    
    faces_coord = detect_face(frame) # detect more than one face
    if len(faces_coord):
        faces = normalize_faces(frame, faces_coord)
        #faces = normalize_intensity(faces)
        for i, face in enumerate(faces): # for each detected face
            
            
            #cv2.imwrite('trainingData/female/picture_BGR5.jpg',face)
            test = pca1.transform(face.reshape(1,-1))    
            #print test
            #transform = test.reshape(1,-1)
            #print transform
            prob=svc1.predict_proba(test)
            confidence = svc1.decision_function(test)
            print (confidence)
            print (prob)
           
            
            
            pred = svc1.predict(test)
            print (pred,pred[0])
           
            name=labels_dic[pred[0]].capitalize()
            print (name)
            
            #pred = labels_dic[pred[0]].capitalize()
            #threshold = .50
            
            if prob[0][1]>.75:
                
                cv2.putText(frame, 'vineet',(faces_coord[i][0], faces_coord[i][1] - 10),
                            cv2.FONT_HERSHEY_PLAIN, 2, (66, 53, 243), 2)
            
                
            elif prob[0][0]>.80:
                cv2.putText(frame,'unknown',(faces_coord[i][0], faces_coord[i][1] - 10),
                            cv2.FONT_HERSHEY_PLAIN, 3, (66, 53, 243), 2)
                
                
            #cv2.putText(frame,name,(x,y-10),font,2,(0,0,255),2,cv2.LINE_AA)
            
               
           #cv2.putText(frame,'Unknown',(x,y-10),font,2,(0,0,255),2,cv2.LINE_AA)
        clear_output(wait = True)
        draw_rectangle(frame, faces_coord) # rectangle around face
        
    cv2.putText(frame, "ESC to exit", (5, frame.shape[0] - 5),cv2.FONT_HERSHEY_PLAIN, 1.3, (66, 53, 243), 2,
                cv2.LINE_AA)
    
    cv2.imshow("opencv_face", frame) # live feed in external
    if cv2.waitKey(5) == 27:
        break
        
cam.release()
cv2.destroyAllWindows()
        